In [2]:
puzzle1 = [
	[4,1,3],
	[2,8,0],
	[7,6,5]
]
puzzle2 = [
	[10, 3, 6, 4],
	[ 1, 5, 8, 0],
	[ 2,13, 7,15],
	[14, 9,12,11]
]
puzzle3 = [
	[ 3, 7,14,15,10],
	[ 1, 0, 5, 9, 4],
	[16, 2,11,12, 8],
	[17, 6,13,18,20],
	[21,22,23,19,24]
]

In [3]:
from copy import deepcopy

shortest_depths = {}

TOP = 0
RIGHT = 1
BOTTOM = 2
LEFT = 3

class SliderState:
    def __init__(self, ar, lastShift = None, history = ""):
        self.grid = ar
        #Decrease backtracking by not allowing the reverse of the move
        self.lastShift = lastShift        
        self.history = history        
        self.setStartingState()
        
    def setStartingState(self):       
        self.n = len(self.grid)
        lastNumber = 0
        self.isWin = True
        for row in range(self.n):
            for col in range(self.n):
                num = self.grid[row][col]
                if(num == 0):
                    self.slider_row = row
                    self.slider_col = col
                    #short circuit the search
                    if(not self.isWin):
                        return
                    continue
                if(lastNumber+1 != num):
                    self.isWin = False
                lastNumber = num
                                
    def getAllTransitions(self):
        '''
        Yields 2 to 4 slider-state clones.
        '''
        for newRC in self.getLegalTransitions():
            newGrid = self.getShiftedGrid(newRC[0],newRC[1])
            movedNumber = self.grid[newRC[0]][newRC[1]]
            yield SliderState(newGrid, newRC[2], self.history+str(movedNumber))
    
    def getLegalTransitions(self):
        #Top
        if(self.slider_row > 0 and self.lastShift != BOTTOM):
            yield self.slider_row-1, self.slider_col, TOP
        #Right
        if(self.slider_col < self.n-1 and self.lastShift != LEFT):
            yield self.slider_row, self.slider_col+1, RIGHT
        #Bottom
        if(self.slider_row < self.n-1 and self.lastShift != TOP):
            yield self.slider_row+1, self.slider_col, BOTTOM
        #Left
        if(self.slider_col > 0 and self.lastShift != RIGHT):
            yield self.slider_row, self.slider_col-1, LEFT
    
    def getShiftedGrid(self, row, col):
        newGrid = deepcopy(self.grid)
        newGrid[row][col], newGrid[self.slider_row][self.slider_col] = newGrid[self.slider_row][self.slider_col], newGrid[row][col]
        return newGrid

In [4]:
def solve(sliderState, remainingDepth):
    if(remainingDepth <= 0):
        return None
    
    if(sliderState.isWin):
        return sliderState
    
    for slide in sliderState.getAllTransitions():
        grid_as_string = str(slide.grid)
        #if it's already been in this state, don't recurse into it
        if(grid_as_string in shortest_depths.keys() and
           shortest_depths[grid_as_string] > remainingDepth):
            shortest_depths[grid_as_string] = remainingDepth
            continue
        shortest_depths[grid_as_string] = remainingDepth
        sol = solve(slide, remainingDepth-1)
        if(sol != None):
            return sol
    
def slide_puzzle(ar):
    global shortest_depths
    shortest_depths = {}
    instance = SliderState(ar)
    for i in range(20):
        print(i)
        sol = solve(instance,i)
        if(sol != None):
            break
    
    if(sol == None):
        print("No solution found")
        return []
    return [int(i) for i in sol.history]

In [ ]:
def solverV2(puzzle):
    start = SliderState(puzzle)
    ends = [[start]]
    #get deeper steps
    for steps in range(0, 50):
        print(steps)
        #add a new layer for the current depth
        ends.append([])    
        for i in range(len(ends[steps])):
            for slider in ends[steps][i].getAllTransitions():
                if(slider.isWin):
                    return slider
                ends[steps+1].append(slider)        
            
print(str(solverV2(puzzle1).history))
print(str(solverV2(puzzle2).history))
print(str(solverV2(puzzle3).history))